In [3]:
import json
from rank_bm25 import BM25Okapi
from tqdm import tqdm

In [7]:
with open('data\corpus_evidence_unified.json', 'r') as f:
    evidence_corpus = json.load(f)

# evidence_corpus[0]

<>:1: SyntaxWarning: invalid escape sequence '\c'
<>:1: SyntaxWarning: invalid escape sequence '\c'
C:\Users\kuipe\AppData\Local\Temp\ipykernel_12024\952399766.py:1: SyntaxWarning: invalid escape sequence '\c'
  with open('data\corpus_evidence_unified.json', 'r') as f:


In [15]:
# Load the claims
with open(r'data\train_claims_quantemp.json', 'r') as f:
    claims = json.load(f)


In [18]:
corpus = [evidence for evidence in tqdm(evidence_corpus.values())]
tokenized_corpus = [doc.split() for doc in corpus]

100%|██████████| 423320/423320 [00:00<00:00, 3066773.01it/s]


In [20]:
bm25 = BM25Okapi(tokenized_corpus)

In [29]:
def retrieve_evidence(claim, k=100):
    tokenized_claim = claim.split()
    scores = bm25.get_scores(tokenized_claim)
    top_k_indices = scores.argsort()[-k:][::-1]
    top_k_documents = [corpus[idx] for idx in top_k_indices]
    return top_k_documents

In [30]:
# Prepare data for NLI model
def prepare_nli_input(claim, evidence):
    nli_input = {
        'claim': claim,
        'evidence': " ".join(evidence)  # Concatenate top-k evidence into a single string
    }
    return nli_input

In [40]:
# claims_subset = claims[:5]
nli_data = []
for claim in tqdm(claims):
    claim_text = claim['claim']
    evidence = retrieve_evidence(claim_text)
    nli_input = prepare_nli_input(claim_text, evidence)
    nli_data.append(nli_input)

  0%|          | 1/9935 [00:02<6:51:46,  2.49s/it]

In [36]:
print(len(nli_data[0]['evidence']))

19778


In [37]:
print(nli_data[0]['claim'])

In her budget speech, Nirmala Sitharaman claimed that the Government distributed 35,000 crore LED bulbs in the country.


In [38]:
with open('data/nli_input.json', 'w') as f:
    json.dump(nli_data, f)

In [39]:
for i, entry in enumerate(nli_data[:5]):
    print(f"Claim {i+1}: {entry['claim']}")
    print(f"Evidence {i+1}: {entry['evidence']}")

Claim 1: In her budget speech, Nirmala Sitharaman claimed that the Government distributed 35,000 crore LED bulbs in the country.
Evidence 1: conclusion: nirmala sitharaman didn't claim that govt distributed 35,000 crores led bulbs. from the video fo the speech it is clear that she said 35 crores. nirmala sitharaman didn't claim that govt distributed 35,000 crores led bulbs. from the video of the speech, it is clear that she said 35 crores, not 35000 crores. 34.7 crore led lamps were distributed to replace icl. . (100w). additionally around ... as on date, 14.05 lakh led bulbs have been distributed across the state ...148 pagine 7 feb 2017  "the government has distributed 21 crore led bulbs and due to this power bills have been reduced and rs 11,000 crore was saved," the prime ... 5 gen 2023  state-owned energy efficiency services ltd (eesl) on thursday said it has distributed over 36.8 crore led bulbs under the government's unnat ... all the 35,000 teachers will be posted to junior sec